In [2]:
import pandas as pd
import pprint
import os
import setuptools
import re
import spacy
from spacy import displacy

In [3]:
# ==== Carga de archivo ====
df = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\corpus\dialogos.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dialogo  35 non-null     object
 1   titulo   35 non-null     object
dtypes: object(2)
memory usage: 692.0+ bytes


In [3]:
# Carga modelo español large
nlp = spacy.load('es_core_news_lg')

c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_lg' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
# Limpieza básica del texto creamos la variable "obras" contenido limpio
def limpiar_texto(texto):
    texto = texto.lower()
    texto = texto.replace('\n', ' ').replace('\r', ' ')
    texto = texto.strip()
    return texto

df["obras"] = df["contenido_escr"].apply(limpiar_texto)
df["contenido_escr_limpio"] = df["obras"]

In [5]:
# Extracción de preguntas explícitas

def extraer_preguntas(texto):
    doc = nlp(texto)
    preguntas = [sent.text for sent in doc.sents if "?" in sent.text or "¿" in sent.text]
    return preguntas

df["preguntas_filosoficas"] = df["obras"].apply(extraer_preguntas)

In [6]:
df["preguntas_filosoficas"].astype

<bound method NDFrame.astype of 0                                                    []
1                                                    []
2     [¿qué es lo justo?, ,¿qué demonio ha inspirado...
3     [hé aquí otra serie de preguntas y respuestas ...
4     [morir es hacerse insensible, es decir, incapa...
5                                                    []
6                                                    []
7                                                    []
8     [por otra parte, ¿cómo puede admitirse esta id...
9     [¿consiste el ser filósofo en querer conocer t...
10                                                   []
11                        [¿quieres hacerte un tirano?]
12                                                   []
13                                                   []
14    [¿no es permitido suponer, que llegó un día, e...
15                                                   []
16                                                   []
17              

In [7]:
    for index, fila in df.iterrows():
        preguntas = fila["preguntas_filosoficas"]
        
        # verificamos si no esta vacio
        if preguntas:
            print(f"Fila {index}:")
            for pregunta in preguntas:
                print(f"  - {pregunta}")
            print("-" * 40)

Fila 2:
  - ¿qué es lo justo?
  - ,¿qué demonio ha inspirado a simón el socrático la idea de escribir diálogos para fastidio de los traductores, puesto que los hay, y de los lectores, suponiendo que los haya?
----------------------------------------
Fila 3:
  - hé aquí otra serie de preguntas y respuestas de un interés literario y filosófico escaso, por no decir nulo, y que se pueden resumir en las tres preguntas y tres respuestas siguientes:,¿la virtud puede por su naturaleza ser enseñada?
  - ,¿será la virtud un don de la naturaleza, es decir, una cualidad natural?
  - ,¿pero si la virtud no procede ni de la enseñanza ni de la naturaleza, de donde viene?
----------------------------------------
Fila 4:
  - morir es hacerse insensible, es decir, incapaz de sufrir; y entonces ¿cómo la muerte puede ser un mal?
  - ¿cuál es la edad que no tiene sus dolores?
  - ¿qué oficio, qué arte, qué función deja de dar lugar a disgustos, percances y desesperaciones?
---------------------------------

In [8]:
# Extracción de preguntas implícitas y respuestas

def preguntas_implicitas_respuestas(texto):
    doc = nlp(texto)
    resultados = []
    ultima_pregunta = None

    for sent in doc.sents:
        frase = sent.text.strip()

        if re.match(r"(?i)^(dime|explícame|quisiera saber|cuéntame|hazme saber)", frase):
            ultima_pregunta = frase
            continue

        if ultima_pregunta and len(frase) > 10:
            resultados.append({
                "tipo": "implícita",
                "pregunta": ultima_pregunta,
                "respuesta": frase
            })
            ultima_pregunta = None

    return resultados


In [9]:
df["preguntas_respuestas_implicitas"] = df["contenido_escr_limpio"].apply(preguntas_implicitas_respuestas)

In [10]:
# Extracción de Entidades (NER) antes de procesar el texto
def extraer_entidades(texto):
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

df["entidades"] = df["obras"].apply(extraer_entidades)

In [11]:
for index, fila in df.iterrows():
    entidades = fila["entidades"]
    
    # Verificamos si no esta vacio
    if entidades:
        print("Fila {index}")
        for entidad in entidades:
            print(f" - {entidades}")
        print("-" *40)

Fila {index}
 - [('clitofon', 'PER'), ('sócrates', 'PER'), ('sócrates', 'PER')]
 - [('clitofon', 'PER'), ('sócrates', 'PER'), ('sócrates', 'PER')]
 - [('clitofon', 'PER'), ('sócrates', 'PER'), ('sócrates', 'PER')]
----------------------------------------
Fila {index}
 - [('dion a dionisio', 'PER'), ('dionisio', 'PER'), ('dion', 'PER'), ('dion', 'PER'), ('dion', 'PER'), ('hermias', 'PER'), ('erasmo', 'PER'), ('coriseo', 'PER'), ('sicilia', 'LOC')]
 - [('dion a dionisio', 'PER'), ('dionisio', 'PER'), ('dion', 'PER'), ('dion', 'PER'), ('dion', 'PER'), ('hermias', 'PER'), ('erasmo', 'PER'), ('coriseo', 'PER'), ('sicilia', 'LOC')]
 - [('dion a dionisio', 'PER'), ('dionisio', 'PER'), ('dion', 'PER'), ('dion', 'PER'), ('dion', 'PER'), ('hermias', 'PER'), ('erasmo', 'PER'), ('coriseo', 'PER'), ('sicilia', 'LOC')]
 - [('dion a dionisio', 'PER'), ('dionisio', 'PER'), ('dion', 'PER'), ('dion', 'PER'), ('dion', 'PER'), ('hermias', 'PER'), ('erasmo', 'PER'), ('coriseo', 'PER'), ('sicilia', 'LOC')]


In [12]:
# Filtrado de Fragmentos Estilo Socrático

def extraer_fragmentos_socraticos(texto):
    lineas = texto.split("\n")
    fragmentos = [linea for linea in lineas if "¿" in linea or ":" in linea]
    return " ".join(fragmentos)

df["fragmentos_socraticos"] = df["obras"].apply(extraer_fragmentos_socraticos)


In [13]:
# Procesamiento de Texto (Lematización y eliminación de stopwords)

def procesar_escrito(texto):
    doc = nlp(texto)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

df["escritos_lematizacion"] = df["obras"].apply(procesar_escrito)

In [14]:
# Combinación de preguntas explícitas e implícitas en un solo campo JSON

def combinar_preguntas(row):
    preguntas = []
    for pregunta in row["preguntas_filosoficas"]:
        preguntas.append({
            "tipo": "explícita",
            "pregunta": pregunta,
            "respuesta": None
        })
    for par in row["preguntas_respuestas_implicitas"]:
        preguntas.append(par)
    return preguntas

df["preguntas_combinadas"] = df.apply(combinar_preguntas, axis=1)


In [15]:
# Imprimir la salida estructurada
for index, fila in df.iterrows():
    titulo = fila["titulo"]
    preguntas_combinadas = fila["preguntas_combinadas"]
    
    # Verificamos si hay preguntas combinadas
    if preguntas_combinadas:
        print(f"📚 Título: {titulo} (Fila {index})")
        for pregunta in preguntas_combinadas:
            tipo_pregunta = pregunta["tipo"]
            pregunta_texto = pregunta["pregunta"]
            respuesta_texto = pregunta["respuesta"] if pregunta["respuesta"] else "No disponible"
            
            print(f"  - Tipo: {tipo_pregunta}")
            print(f"    Pregunta: {pregunta_texto}")
            print(f"    Respuesta: {respuesta_texto}")
        print("-" * 60)

📚 Título: De lo justo (Fila 2)
  - Tipo: explícita
    Pregunta: ¿qué es lo justo?
    Respuesta: No disponible
  - Tipo: explícita
    Pregunta: ,¿qué demonio ha inspirado a simón el socrático la idea de escribir diálogos para fastidio de los traductores, puesto que los hay, y de los lectores, suponiendo que los haya?
    Respuesta: No disponible
------------------------------------------------------------
📚 Título: De la virtud (Fila 3)
  - Tipo: explícita
    Pregunta: hé aquí otra serie de preguntas y respuestas de un interés literario y filosófico escaso, por no decir nulo, y que se pueden resumir en las tres preguntas y tres respuestas siguientes:,¿la virtud puede por su naturaleza ser enseñada?
    Respuesta: No disponible
  - Tipo: explícita
    Pregunta: ,¿será la virtud un don de la naturaleza, es decir, una cualidad natural?
    Respuesta: No disponible
  - Tipo: explícita
    Pregunta: ,¿pero si la virtud no procede ni de la enseñanza ni de la naturaleza, de donde viene?
  

In [16]:
# Exportar a JSON estructurado
json_data = []
for _, fila in df.iterrows():
    json_data.append({
        "ID_obra": fila["ID_obra"],
        "titulo": fila["titulo"],
        "preguntas": fila["preguntas_combinadas"],
        "entidades": fila["entidades"],
        "fragmentos_socraticos": fila["fragmentos_socraticos"],
        "escritos_lematizacion": fila["escritos_lematizacion"]
    })

In [17]:
output_json = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed\obras_filosoficas_struct.json"
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

In [18]:
output_directory = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed"

# Guardar el DataFrame procesado
df.to_csv(os.path.join(output_directory, "obras_socraticos_procesados.csv"), index=False)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   ID_obra                          51 non-null     int64 
 1   titulo                           51 non-null     object
 2   contenido_escr                   51 non-null     object
 3   obras                            51 non-null     object
 4   contenido_escr_limpio            51 non-null     object
 5   preguntas_filosoficas            51 non-null     object
 6   preguntas_respuestas_implicitas  51 non-null     object
 7   entidades                        51 non-null     object
 8   fragmentos_socraticos            51 non-null     object
 9   escritos_lematizacion            51 non-null     object
 10  preguntas_combinadas             51 non-null     object
dtypes: int64(1), object(10)
memory usage: 4.5+ KB
